In [25]:
using DynamicPolynomials, LinearAlgebra

@ncpolyvar x1 x2 x3 x4 x5 y1 y2 y3 y4 y5

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function get_star_alg(f)
    mom=monomials(f)
    coe=coefficients(f)
    mom=star_algebra.(mom)
    return dot(mom,coe)
end


X = [x1; x2; x3; x4; x5];
Y = [y1; y2; y3; y4; y5];
var=Vector{PolyVar{false}}([X;Y])
C = [-1 -1 0 -1 1; -1 -1 -1 1 0; 1 1 -1 1 1; -1 1 0 0 0; 0 0 1 1 0];

A = [0 0 -1 0 -1]; B = [0 -1 0 -1 -1];

u =A*X +B*Y + X'*C*Y;

u = x1*(y1+y2+y3)+x2*(y1+y2-y3)+x3*(y1-y2)-x1-2*y1-y2

f = -0.5*(u + get_star_alg(u))

g=Vector{Polynomial{false,Float64}}([])

h=1.0*[X[j]-X[j]^2 for j in 1:5]
h=[h;[Y[j]-Y[j]^2 for j in 1:5]]
h=[h;[X[j]*Y[i]-Y[i]*X[j] for i in 1:5 for j in 1:5]]

f=f(var=>(0.5*var.+0.5))

for j in 1:length(h)
    h[j]=4*h[j](var=>(0.5*var.+0.5))
end

In [26]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(var,f,g,h);

In [27]:
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

Number of variables: n=10
Number of inequality constraints: m=0
Number of equality constraints: l=35


In [28]:
k=3

println("Relaxation order: k=",k)

Relaxation order: k=3


In [29]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-3,
                                 use_eqcons_to_get_constant_trace=true,
                                 check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 1111.0
  Number of blocks: omega=

1
  Size of the largest block: s^max=1111
  Number of equality trace constraints: zeta=256586
Modeling time:
231.071087 seconds (63.72 M allocations: 1.387 TiB, 6.69% gc time)
iter=1   val=-277.75000000000006   gap=8.156903154914153e-16   feas=2730.8161859903107
iter=2   val=-71.70677928754719   gap=0.7043630140553391   feas=1894.9364668850974
iter=4   val=-148.4255031905788   gap=0.2487494086284198   feas=1444.8114694884816
iter=8   val=-39.476126002412435   gap=0.708356058064746   feas=734.4263864313411
iter=16   val=-11.107721779776696   gap=0.9095925229111842   feas=405.58464007369736
iter=32   val=-3.046716081327713   gap=0.9661036130429298   feas=358.52012929473375
iter=64   val=-1.5531952148682708   gap=0.9714386490759824   feas=110.5921617548937
iter=128   val=-0.8406720732917424   gap=0.9745644932816284   feas=45.77914984564275
iter=256   val=-0.37253873967641743   gap=0.963224342294251   feas=45.59330361314494
iter=512   val=0.1197680252993557   gap=0.9347034311803599   feas=

InterruptException: InterruptException:

In [ ]:
testing Bell ineq start: 3h00 stop:12h39

In [31]:
9*3600+39*60

34740

In [30]:
using NCTSSOS

@time ctpNCPOP.nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
    n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS=false,QUIET=false);

------------------------------------------------------
The sizes of blocks:
[1111]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 556666          
  Cones                  : 0               
  Scalar variables       : 217561          
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.10            
Lin. dep.  - nu

Mosek.MosekError: Mosek.MosekError(1051, "Out of space.")